## Prerequisite

After compiling the DFT driver and installing PyCDFT, run the ground state calculation.
- - - - - - -
For Qbox

```bash
 export qb="/path/to/executable"
 $qb < gs.in > gs.out
```
Then in the same directory, run [Qbox in server mode](qboxcode.org/daoc/html/usage/client-server.html) (using interactive queue)
    

```bash
 mpirun -np <ntasks> $qb -server qb_cdft.in qb_cdft.out
```

where qb_cdft.\* are files reserved in client-server mode.

### Tutorial: coupling constant for thiophene

In [ ]:
from pycdft import *
from ase.io import read

In [ ]:
cell = read("./REPL1")
print(r"Initial atomic positions (Ang):")
print(cell.get_positions())
print(r"Initial lattice parameters (Ang):")
print(cell.get_cell())

In [ ]:
V = (-0.8,0.8)
#V = -0.135568
    
print("==================== Initializing Run =========================")
# load sample geometry
sample = Sample(ase_cell=cell, n1=REPLX, n2=REPLY, n3=REPLZ, vspin=1)
print(sample.atoms[1])
    
# load DFT driver
qboxdriver = QboxDriver(
    sample=sample,
    init_cmd="load gs.xml\n"
             "set xc PBE\n"
             "set wf_dyn JD\n"
             "set scf_tol 1.0E-8\n",
    scf_cmd="run 0 50 5"
)
    
# set up CDFT constraints and solver
solver1 = CDFTSolver(job="scf", optimizer="brentq",sample=sample, dft_driver=qboxdriver)
solver2 = solver1.copy()
    
# add constraint to two solvers
ChargeTransferConstraint(
    sample=solver1.sample,
    donor=Fragment(solver1.sample, solver1.sample.atoms[0:9]),
    acceptor=Fragment(solver1.sample, solver1.sample.atoms[9:18]),
    V_brak=V,
    #V_init = V,
    N0=1,
    N_tol=5E-5
)
ChargeTransferConstraint(
    sample=solver2.sample, 
    donor=Fragment(solver2.sample, solver2.sample.atoms[0:9]),
    acceptor=Fragment(solver2.sample, solver2.sample.atoms[9:18]),
    V_brak=V,
    #V_init= -V,
    N0=-1, 
    N_tol=5E-5
)
    
print("~~~~~~~~~~~~~~~~~~~~ Applying CDFT ~~~~~~~~~~~~~~~~~~~~")
print("---- solver A ------")
solver1.solve()
print("---- solver B ------")
solver2.solve()
    
print("~~~~~~~~~~~~~~~~~~~~ Calculating coupling ~~~~~~~~~~~~~~~~~~~~")
compute_elcoupling(solver1, solver2)
#print("~~~~~~~~~~~~~~~~~~~~ Calculating coupling (nonsymm comparison) ~~~~~~~~~~~~~~~~~~~~")
#compute_elcoupling_nonsymm(solver1, solver2)
    
print("==================== JOB DONE =========================")